# MTTN to flat TTNs
## Input: MTTN created in 'ttn_to_mttn'
## Output: One flat TTN for each layer in MTTN

In [1]:
import uunet.multinet as ml
import pandas as pd
import re

In [2]:
def initialise_flat_net(ttn,directed,flat_layer_name):
    '''
    initialises a one-layer net.
    actors in flat_net gets same attributes as message-vertices in ttn + attrubyte "type" 
    '''
    # create empty net
    flat_net = ml.empty()

    # add layer
    ml.add_layers(flat_net, [flat_layer_name], directed=[directed])

    # get attributes from ttn
    attributes = ml.attributes(ttn, target='vertex')

    # add attributes from ttn to flat_net
    for attribute,type_ in zip(attributes['name'],attributes['type']):
        if type_ != "string":
            type_ = "numeric"
        ml.add_attributes(flat_net, [attribute], target = "actor", type=type_)

    # add attribute type to flat_net
    ml.add_attributes(flat_net, ["type"], target = "actor", type="string")

    return flat_net

def clean_names(name):
    return re.sub('_\w+', '', name)
    
def flatten_net(ttn,directed,flat_layer_name,layer_to_collapse):
    '''
    flattens ttn to a one-layer network
    '''
    # initialise flat network
    flat_net = initialise_flat_net(ttn,directed,flat_layer_name)
    
    # get actors (messages and person) from ttn
    persons = ml.actors(ttn, layers=['persons'])['actor']
    messages = ml.actors(ttn, layers=[layer_to_collapse])['actor']
    clean_messages = list(map(clean_names,messages))
    all_actors = persons + clean_messages

    # add persons and messages to net
    ml.add_actors(flat_net, all_actors)

    # get attributes of ttn 
    attributes = ml.attributes(ttn, target='vertex') # attributes of ttn
    
    # set attributes for persons
    flat_persons_actors = {"actor": persons, "layer": [flat_layer_name]*len(persons)}
    for a,type_ in zip(attributes['name'],attributes['type']):
        if type_ == "string":
            values = ["-"] * len(persons)
        elif type_ == "double":
            values = [-1] * len(persons)
        ml.set_values(flat_net, a, actors = flat_persons_actors, values = values)

    # break if there is no messages
    if len(clean_messages) == 0:
        return flat_net

    # get attributes message attributes and add to messages in flat ttn
    ttn_message_vertices = {"actor": clean_messages, "layer": ['messages']*len(clean_messages)} # vertices for getting attributes from ttn
    flat_message_actors = {"actor": clean_messages, "layer": [flat_layer_name]*len(clean_messages)} # actors for setting message attributes for flat_net
    for a in attributes['name']:
        values = ml.get_values(ttn, a, vertices = ttn_message_vertices)[a] # get values from ttn messages layer
        ml.set_values(flat_net, a, actors = flat_message_actors, values = values) # add values to flat_net

    
    # set type-attribute to distinguish between persons and messages
    flat_person_actors = {"actor": persons, "layer": [flat_layer_name]*len(clean_messages)} # actors for setting person attributes for flat_net
    ml.set_values(flat_net, "type", actors = flat_message_actors, values = [layer_to_collapse]*len(clean_messages)) # set messages as type message
    ml.set_values(flat_net, "type", actors = flat_person_actors, values = ['person']*len(persons)) # set persons as type person

    # get all edges from ttn
    ttn_edges = ml.edges(ttn, layers1=['persons',layer_to_collapse])

    # construct edges for flat_net
    from_actors  = ttn_edges['from_actor']
    to_actors = ttn_edges['to_actor']

    # clean names 
    clean_from_actors = list(map(clean_names, from_actors))
    clean_to_actors = list(map(clean_names, to_actors))

    # construct edges
    flat_edges = {"from_actor": clean_from_actors, "from_layer": [flat_layer_name]*len(clean_from_actors), 
                         "to_actor": clean_to_actors, "to_layer": [flat_layer_name]*len(clean_to_actors)}

    # add all edges to flat network
    ml.add_edges(flat_net, flat_edges)

    return flat_net

<>:27: SyntaxWarning: invalid escape sequence '\w'
<>:27: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Asus\AppData\Local\Temp\ipykernel_23028\1358833850.py:27: SyntaxWarning: invalid escape sequence '\w'
  return re.sub('_\w+', '', name)


In [3]:
# load ttn
ttn = ml.read('data/ttn_nets/mttn.txt')

# get layers in net
layers = ml.layers(ttn)
layers.remove('persons')

# name of flat layer
flat_layer_name = 'p_and_m'
network_types = [{'directed':True},{'directed':False}]

for network_type in network_types:

    # unpack network type
    directed = network_type['directed']

    if directed:
        name_base = 'flatt_ttn_directed'
    else:
        name_base = 'flatt_ttn_undirected'
        
    # create one flat network for each layer
    for layer_to_collapse in layers:

        flat_net = flatten_net(ttn,directed,flat_layer_name,layer_to_collapse)
        len(ml.actors(ttn)['actor'])
        # save ttn (both graphml and multilayer)
        save_as = f'data/flat_ttn_nets/{name_base}_{layer_to_collapse}'
        ml.write(n = flat_net,file = save_as+'.graphml', format = "graphml")
        ml.write(n = flat_net,file = save_as+'.txt', format = "multilayer")
        
        #display(pd.DataFrame.from_dict(ml.edges(flat_net)))

hejssan


<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Asus\AppData\Local\Temp\ipykernel_17800\1097882755.py:2: SyntaxWarning: invalid escape sequence '\w'
  new_text = re.sub('\w+_', '', text)
